In [4]:
import os

basedir = '/orange/pinaki.sarder/ahmed.naglah/inflammation_project/qp_projects/QP_23SP-25338'

landmark_ihc_file = f'{basedir}/105706_landmarks.geojson'
landmark_he_file = f'{basedir}/105705_landmarks.geojson'

ihc_anno_file = f'{basedir}/105706_dsa.json'

he_anno_file_output = f'{basedir}/105705_transformed_dsa.json'

In [2]:
import json
import numpy as np
import cv2

# Function to load landmark points from GeoJSON
def load_landmarks(geojson_path):
    with open(geojson_path, 'r') as f:
        data = json.load(f)
    
    landmarks = {}
    for feature in data:
        name = feature["properties"]["name"]
        coordinates = feature["geometry"]["coordinates"]
        landmarks[name] = coordinates  # Store as {name: [x, y]}
    
    return landmarks

# Load landmarks from two GeoJSON files
landmarks_img1 = load_landmarks(landmark_ihc_file)
landmarks_img2 = load_landmarks(landmark_he_file)

# Find matching points based on the 'name' field
common_names = set(landmarks_img1.keys()) & set(landmarks_img2.keys())

if len(common_names) < 3:
    raise ValueError("At least 3 matching points are required for affine transformation.")

# Extract corresponding points
src_pts = np.array([landmarks_img1[name] for name in common_names], dtype=np.float32)
dst_pts = np.array([landmarks_img2[name] for name in common_names], dtype=np.float32)

# Compute the affine transformation matrix
affine_matrix, _ = cv2.estimateAffinePartial2D(src_pts, dst_pts)

In [3]:
print(affine_matrix)

[[ 1.01413744e+00 -6.92025918e-02  2.27428658e+03]
 [ 6.92025918e-02  1.01413744e+00 -2.65051162e+03]]


In [5]:
import json
import numpy as np
import cv2

# Function to apply affine transformation to a set of points
def apply_affine_transformation(points, affine_matrix):
    points_array = np.array(points, dtype=np.float32)[:, :2]  # Ignore the third dimension (Z)
    transformed_points = cv2.transform(np.array([points_array]), affine_matrix)[0]
    transformed_points = np.hstack((transformed_points, np.zeros((transformed_points.shape[0], 1))))  # Restore third dimension
    return transformed_points.tolist()

# Function to load JSON annotation file
def load_annotations(json_path):
    with open(json_path, 'r') as f:
        return json.load(f)

# Function to save the transformed annotations back to JSON
def save_annotations(json_path, annotations):
    with open(json_path, 'w') as f:
        json.dump(annotations, f, indent=4)

# Load original annotations
annotations = load_annotations(ihc_anno_file)

# Apply transformation to all annotation points
for annotation in annotations:
    for element in annotation["elements"]:
        if "points" in element:
            element["points"] = apply_affine_transformation(element["points"], affine_matrix)

# Save the transformed annotations
save_annotations(he_anno_file_output, annotations)

print("Transformation applied and saved to transformed_annotations.json")

Transformation applied and saved to transformed_annotations.json
